In [1]:
import pandas as pd
import torch
import math
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from tqdm import tqdm
from scipy.stats import entropy
import random

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('validation.csv')
test_df = pd.read_csv('test.csv')
T_TTA_test_df = pd.read_csv('T_TTA_enhanced_test.csv')
R_TTA_test_df = pd.read_csv('R_TTA_enhanced_test.csv')
MSTTA_test_df = pd.read_csv('MSTTA_enhanced_test.csv')

class SentimentDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        sentence = self.dataframe.iloc[index]['sentence']
        label = self.dataframe.iloc[index]['label']
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True,
        )
        return {
            'sentence_text': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

def create_data_loader(df, tokenizer, max_len, batch_size,shuffle):
    ds = SentimentDataset(
        dataframe=df,
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle = shuffle
    )


PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels=5)

for param in model.bert.parameters():
    param.requires_grad = False

for param in model.bert.encoder.layer[-3:].parameters():
    param.requires_grad = True
    
for param in model.classifier.parameters():
    param.requires_grad = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
model = model.to(device)

max_len = 128
batch_size = 32

train_data_loader = create_data_loader(train_df, tokenizer, max_len, batch_size,shuffle = True)
val_data_loader = create_data_loader(val_df, tokenizer, max_len, batch_size,shuffle = False)
test_data_loader = create_data_loader(test_df, tokenizer, max_len, batch_size,shuffle = False)
T_TTA_test_data_loader = create_data_loader(T_TTA_test_df, tokenizer, max_len, batch_size,shuffle = False)
R_TTA_test_data_loader = create_data_loader(R_TTA_test_df, tokenizer, max_len, batch_size,shuffle = False)
MSTTA_test_data_loader= create_data_loader(MSTTA_test_df, tokenizer, max_len, batch_size,shuffle = False)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

True


In [6]:

for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")

optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5, correct_bias=False)
total_steps = len(train_data_loader) * 20  

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = torch.nn.CrossEntropyLoss().to(device)

def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in tqdm(data_loader, desc="Training"):
        input_ids = d['input_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)
        labels = d['label'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs.logits, dim=1)
        loss = loss_fn(outputs.logits, labels)

        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in tqdm(data_loader, desc="Evaluating"):
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            labels = d['label'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs.logits, dim=1)
            loss = loss_fn(outputs.logits, labels)

            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

best_loss = np.inf
patience = 3 
counter = 0  

for epoch in range(20):
    print(f'Epoch {epoch + 1}/20')
    print('-' * 50)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(train_df)
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device,
        len(val_df)
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')

    if val_loss < best_loss:
        torch.save(model.state_dict(), 'Compare_model.pth')
        best_loss = val_loss
        counter = 0  
    else:
        counter += 1  

    if counter >= patience:
        print("Early stopping triggered")
        break


bert.embeddings.word_embeddings.weight: False
bert.embeddings.position_embeddings.weight: False
bert.embeddings.token_type_embeddings.weight: False
bert.embeddings.LayerNorm.weight: False
bert.embeddings.LayerNorm.bias: False
bert.encoder.layer.0.attention.self.query.weight: False
bert.encoder.layer.0.attention.self.query.bias: False
bert.encoder.layer.0.attention.self.key.weight: False
bert.encoder.layer.0.attention.self.key.bias: False
bert.encoder.layer.0.attention.self.value.weight: False
bert.encoder.layer.0.attention.self.value.bias: False
bert.encoder.layer.0.attention.output.dense.weight: False
bert.encoder.layer.0.attention.output.dense.bias: False
bert.encoder.layer.0.attention.output.LayerNorm.weight: False
bert.encoder.layer.0.attention.output.LayerNorm.bias: False
bert.encoder.layer.0.intermediate.dense.weight: False
bert.encoder.layer.0.intermediate.dense.bias: False
bert.encoder.layer.0.output.dense.weight: False
bert.encoder.layer.0.output.dense.bias: False
bert.encoder

Training: 100%|██████████████████████████████████████████████████████████████████████| 267/267 [00:43<00:00,  6.17it/s]


Train loss 1.261708624354016 accuracy 0.4479166666666667


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 35/35 [00:03<00:00,  9.12it/s]


Val   loss 1.1736017431531633 accuracy 0.4786557674841053
Epoch 2/20
--------------------------------------------------


...

Val   loss 1.1477435946464538 accuracy 0.5068119891008174
Epoch 5/20
--------------------------------------------------


Training: 100%|██████████████████████████████████████████████████████████████████████| 267/267 [00:40<00:00,  6.58it/s]


Train loss 0.9894544212112713 accuracy 0.5750234082397003


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 35/35 [00:03<00:00,  9.85it/s]

Val   loss 1.140216486794608 accuracy 0.514078110808356
Early stopping triggered


In [2]:
def predict_probabilities(model, data_loader,device):
    model.eval()
    sentence_probs = []

    with torch.no_grad():
        for d in tqdm(data_loader, desc="Predicting"):
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            sentence_probs.extend(probs.cpu().numpy())

    return np.array(sentence_probs)

model.load_state_dict(torch.load('Compare_model.pth'))
true_labels = test_df['label'].values

simple_preds = []

with torch.no_grad():
    for d in tqdm(test_data_loader, desc="Predicting Simple"):
        input_ids = d['input_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        simple_preds.extend(preds.cpu().numpy())

simple_accuracy = accuracy_score(true_labels, simple_preds)
print(f'Simple Accuracy: {simple_accuracy}')

def calculate_uncertainty_weights(predictions):
    mean_prediction = np.mean(predictions, axis=0)
    variance = np.var(predictions, axis=0) + 1e-10 
    uncertainties = []

    for prediction in predictions:
        diff_square = (prediction - mean_prediction) ** 2
        temp_var = 0.5 * np.log(2 * math.pi * variance)
        temp_max = np.maximum(0, temp_var)
        final_uncertainty = temp_max + (diff_square / (math.sqrt(3) * variance))
        uncertainties.append(final_uncertainty)

    weights = 1 / (np.array(uncertainties) + 1e-10)
    return weights / weights.sum(axis=0)  

def get_grouped_probs(data_loader, model,device):
    sentence_probs = predict_probabilities(model, data_loader,device)
    return [sentence_probs[i:i+3] for i in range(0, len(sentence_probs), 3)]

def compute_weighted_predictions(grouped_probs, weight_type="average"):
    weighted_probs = []
    for group in grouped_probs:
        if weight_type == "average":
            weights = np.array([1/3, 1/3, 1/3])
        elif weight_type == "uncertainty":
            weights = calculate_uncertainty_weights(group)
        else:
            raise ValueError(f"Unknown weight type: {weight_type}")
        weighted_probs.append(np.average(group, axis=0, weights=weights))
    return np.array(weighted_probs)

test_simple_probs = predict_probabilities(model, test_data_loader,device)
T_TTA_grouped_probs = get_grouped_probs(T_TTA_test_data_loader, model,device)
R_TTA_grouped_probs = get_grouped_probs(R_TTA_test_data_loader, model,device)
MSTTA_grouped_probs = get_grouped_probs(MSTTA_test_data_loader, model,device)

simple_preds = np.argmax(test_simple_probs, axis=1)
simple_accuracy = accuracy_score(true_labels, simple_preds)
simple_precision = precision_score(true_labels, simple_preds, average='weighted')
simple_recall = recall_score(true_labels, simple_preds, average='weighted')
simple_f1 = f1_score(true_labels, simple_preds, average='weighted')
print(f"Simple Accuracy: {simple_accuracy:.4f}, Precision: {simple_precision:.4f}, Recall: {simple_recall:.4f}, F1: {simple_f1:.4f}")

weight_types = ["average", "uncertainty"]
results = {}

for weight_type in weight_types:
    T_TTA_weighted_probs = compute_weighted_predictions(T_TTA_grouped_probs, weight_type=weight_type)
    R_TTA_weighted_probs = compute_weighted_predictions(R_TTA_grouped_probs, weight_type=weight_type)
    MSTTA_weighted_probs = compute_weighted_predictions(MSTTA_grouped_probs, weight_type=weight_type)
 
    T_TTA_preds = np.argmax(T_TTA_weighted_probs, axis=1)
    R_TTA_preds = np.argmax(R_TTA_weighted_probs, axis=1)
    MSTTA_preds = np.argmax(MSTTA_weighted_probs, axis=1)
    
    true_labels = test_df['label'].values
    results[f"T_TTA {weight_type}"] = {
        "accuracy": accuracy_score(true_labels, T_TTA_preds),
        "f1": f1_score(true_labels, T_TTA_preds, average='weighted'),
        "precision": precision_score(true_labels, T_TTA_preds, average='weighted'),
        "recall": recall_score(true_labels, T_TTA_preds, average='weighted')
    }
    results[f"R_TTA {weight_type}"] = {
        "accuracy": accuracy_score(true_labels, R_TTA_preds),
        "f1": f1_score(true_labels, R_TTA_preds, average='weighted'),
        "precision": precision_score(true_labels, R_TTA_preds, average='weighted'),
        "recall": recall_score(true_labels, R_TTA_preds, average='weighted')
    }
    results[f"MSTTA {weight_type}"] = {
        "accuracy": accuracy_score(true_labels, MSTTA_preds),
        "f1": f1_score(true_labels, MSTTA_preds, average='weighted'),
        "precision": precision_score(true_labels, MSTTA_preds, average='weighted'),
        "recall": recall_score(true_labels, MSTTA_preds, average='weighted')
    }

for method, metrics in results.items():
    print(f"{method} - Accuracy: {metrics['accuracy']:.4f}, Precision: {metrics['precision']:.4f}, Recall: {metrics['recall']:.4f}, F1: {metrics['f1']:.4f}")


Predicting Simple: 100%|███████████████████████████████████████████████████████████████| 70/70 [00:08<00:00,  8.57it/s]


Simple Accuracy: 0.5180995475113123


Predicting: 100%|████████████████████████████████████████████████████████████████████| 208/208 [00:21<00:00,  9.77it/s]


Simple Accuracy: 0.5181, Precision: 0.5178, Recall: 0.5181, F1: 0.4931
T_TTA average - Accuracy: 0.5149, Precision: 0.5176, Recall: 0.5149, F1: 0.4882
R_TTA average - Accuracy: 0.5149, Precision: 0.5257, Recall: 0.5149, F1: 0.4864
MSTTA average - Accuracy: 0.5271, Precision: 0.5397, Recall: 0.5271, F1: 0.4998
T_TTA uncertainty - Accuracy: 0.5145, Precision: 0.5147, Recall: 0.5145, F1: 0.4892
R_TTA uncertainty - Accuracy: 0.5195, Precision: 0.5259, Recall: 0.5195, F1: 0.4921
MSTTA uncertainty - Accuracy: 0.5249, Precision: 0.5301, Recall: 0.5249, F1: 0.4993
